In [15]:
from time import time
import pandas as pd
import dask.dataframe as dd
import os


## Measuring the Performances of pandas and dask for csv reading

In [9]:
start = time()
df = pd.read_csv("yellow_tripdata_2015-01.csv")
end  = time()
print("loading time is", "{:.6f}".format(end-start), "secs")

460.274211 secs


In [14]:
start = time()
ddf = dd.read_csv("yellow_tripdata_2015-01.csv")
end  = time()
print("loading time is", "{:.6f}".format(end-start), "secs")

loading time is 0.026928 secs


Dask is a lot faster than the pandas

In [21]:
print(df.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')


## Cleaning The Columns

In [6]:
# Validation on columns lowering, stripping, removing white spaces and special characters
df.columns = df.columns.str.lower()
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(" ", "")
df.columns = df.columns.str.replace("\W", "", regex = True)

 

## Creating the YAML file

In [8]:
%%writefile file.yaml
file_type: csv
dataset_name: Yellow Cab Data
file_name: yellow_tripdata_2015-01
inbound_delimiter: ","
outbound_delimiter: "\n"
skip_leading_rows: 0
columns: 
    - VendorID
    - tpep_pickup_datetime
    - tpep_dropoff_datetime
    - passenger_count
    - trip_distance
    - pickup_longitude
    - pickup_latitude
    - RateCodeID
    - store_and_fwd_flag
    - dropoff_longitude
    - dropoff_latitude
    - payment_type
    - fare_amount
    - extra
    - mta_tax
    - tip_amount
    - tolls_amount
    - improvement_surcharge
    - total_amount
      

Overwriting file.yaml


## Validation Script

In [2]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


## Reading the config data

In [9]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [10]:

config_data

{'file_type': 'csv',
 'dataset_name': 'Yellow Cab Data',
 'file_name': 'yellow_tripdata_2015-01',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['VendorID',
  'tpep_pickup_datetime',
  'tpep_dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'pickup_longitude',
  'pickup_latitude',
  'RateCodeID',
  'store_and_fwd_flag',
  'dropoff_longitude',
  'dropoff_latitude',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount']}

## Reading the Csv in a "not hard-coded" manner

In [5]:
file_type = config_data["file_type"]
source_file = "./" + config_data["file_name"] + f".{file_type}"

df = pd.read_csv(source_file, delimiter=config_data["inbound_delimiter"])
df.head() 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


## Validating the columns of dataframe and the config data

In [11]:
util.col_header_val(df, config_data)

column name and column length validation passed


1

In [13]:
df.to_csv("yellow_tripdata_2015-01.csv.gz",
          index=False,
          sep = ("|"),
          compression="gzip",
          )

## Printing the size, 

In [16]:
print("Number of Rows are", len(df))
print("Number of Columns are", len(df.columns))
print("Size of the zipped file is ", os.path.getsize("yellow_tripdata_2015-01.csv.gz"))

Number of Rows are 12748986
Number of Columns are 19
Size of the zipped file is  516819513
